In [3]:
import requests
import time 
import pandas as pd
 
from bs4 import BeautifulSoup

In [4]:

url = "https://www.ss.com/en/agriculture/agricultural-machinery/tractors/"
url

'https://www.ss.com/en/agriculture/agricultural-machinery/tractors/'

In [12]:
req = requests.get(url) 
req.status_code

200

In [13]:
soup = BeautifulSoup(req.text, 'lxml') 

soup.title

<title>SS.COM Agricultural machinery - Tractors, Prices - Advertisements</title>

In [7]:
def getColList(soup):
    column_list = ["description","url"] 
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] 
    column_list += headcolumns
    return column_list

In [14]:
column_names = getColList(soup)
column_names

['description', 'url', 'Brand', 'Year', 'Price']

In [8]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [15]:
def getRow(row, colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [20]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]

In [21]:
def getDFfromURL(url):
    
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None 
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows, columns=column_names)

In [22]:
tractors = "https://www.ss.com/en/agriculture/agricultural-machinery/tractors/"
idf = getDFfromURL(tractors)
idf.head()

,description,url,Brand,Year,Price
0,Fastraks 14t pašmassa. Atrums pa ceļõu 80 km ....,https://ss.com/msg/en/agriculture/agricultural...,Jsb7270,2012,"60,000 €"
1,Netto cena 25000 Eur. \r\nPārdodu ļoti labu tr...,https://ss.com/msg/en/agriculture/agricultural...,CaseMx135,2000,"30,250 €"
2,"Pārdod uzņēmums, eur 6900 + Pvn. 12 cilindru m...",https://ss.com/msg/en/agriculture/agricultural...,KirovecK-701,1991,"8,349 €"
3,Netto cena 35000Eur. \r\nPārdodu ļoti labu tra...,https://ss.com/msg/en/agriculture/agricultural...,ValtraT151,2009,"42,350 €"
4,Pārdodu normālu traktoru darba kārtībā. Kādam ...,https://ss.com/msg/en/agriculture/agricultural...,BelarusMtz,1995,"2,500 €"


In [27]:
idf.to_csv("traktoru_pardosana.csv")  # ... and here I get error in opening the created .csv file and as a result error for the further step

In [33]:
traktori = pd.read_csv('C:\Users\zirav\Desktop\Accenture\Day20_WebScraping\traktoru_pardosana.csv')
traktori

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3926062166.py, line 1)